In [29]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize, least_squares
from dataclasses import dataclass
from typing import Iterable

In [2]:
class Dataset:
    def __init__(self, xk, yk):
        self.xk = xk
        self.yk = yk
        
    def affichage(self, repere, couleur="red"):
        repere.scatter(self.xk, self.yk, color=couleur, label="échantillon")



In [3]:
class Realite:
    def __init__(self, L):
        self.L = L
        
    def __call__(self, xs) -> float:
        return np.sin(10 * np.pi * xs / self.L) / (1 + np.exp(3 * xs / self.L))
    
    def affichage(self, repere, couleur="blue"):
        xs = np.linspace(0, self.L, 500)
        ys = self(xs)
        repere.plot(xs, ys, label="cible", color=couleur)

    def ds_simpliste(self, N: int) -> Dataset:
        xk = np.linspace(0, self.L, N+1)
        yk = self(xk)
        return Dataset(xk=xk, yk=yk)

    def ds_simple(self, N: int, epsilon: float) -> Dataset:
        xk = np.linspace(0, self.L, N+1)
        yk = self(xk) + epsilon * np.random.randn(N+1)
        return Dataset(xk=xk, yk=yk)

    def ds_raisonnable(self, N: int, epsilon: float) -> Dataset:
        xk = np.random.uniform(low=0., high=self.L, size=(N+1, ))
        yk = self(xk) + epsilon * np.random.randn(N+1)
        return Dataset(xk=xk, yk=yk)


In [22]:
class Gaussienne:
    def __init__(self, degre: int, L: float):
        if degre < 0:
            raise ValueError("Le degré doit ^etre positif")
        self.degre = degre
        self.coefficients = np.zeros(shape=(degre+1, ))
        self.L = L
        self.points = np.linspace(0, L, degre+1)
        self.phi = lambda x: np.exp(-(x * degre / L) ** 2)
        
    def __repr__(self):
        return f"Gaussienne(degre={self.degre}, L={self.L})"
        
    def __call__(self, x):
        return sum(
            coef * self.phi(x - point) 
            for coef, point in zip(self.coefficients, self.points)
        )
        
    def affichage(self, repere, couleur="magenta"):
        xs = np.linspace(0, self.L, 500)
        ys = self(xs)
        repere.plot(xs, ys, label="prédicteur", color=couleur)
        
    def erreur_quadratique(self, ds: Dataset) -> float:
        return np.sum((self(ds.xk) - ds.yk)**2) / len(ds.xk)
    
    def entrainement(self, ds: Dataset):
        def calcule_residus(cs):
            self.coefficients = cs
            return self(ds.xk) - ds.yk
        
        def calcule_jac(cs):
            tableau = [
                [
                  self.phi(xi - betaj)
                    for betaj in self.points
                ]
                for xi in ds.xk
            ]
            return np.array(tableau)
        
        opti_result = least_squares(
            fun=calcule_residus,
            jac=calcule_jac,
            x0=self.coefficients,
        )
        if not opti_result.success:
            print("Entrainement Sans succès",)
            print(opti_result.message)
        self.coefficients = opti_result.x



In [9]:
def train_test_split(ds: Dataset, proportion=0.7) -> tuple[Dataset, Dataset]:
    xtest, ytest, xtrain, ytrain = list(), list(), list(), list()
    for x, y in zip(ds.xk, ds.yk):
        if np.random.random() < proportion:
            xtrain.append(x)
            ytrain.append(y)
        else:
            xtest.append(x)
            ytest.append(y)
    return Dataset(xtrain, ytrain), Dataset(xtest, ytest)


In [5]:
@dataclass
class Score:
    modele: Gaussienne
    score_entrainement: float
    score_selection: float


In [18]:
L = 5.
taille_echantillon = 200

reel = Realite(L=L)
ds = reel.ds_raisonnable(N=taille_echantillon, epsilon=0.1)



In [19]:
ds_reste, ds_test = train_test_split(ds, proportion=0.8)
ds_train, ds_select = train_test_split(ds_reste, proportion=0.8)


In [23]:
%%time
scores = list()

for degre in range(10, 100, 3):
    gaussienne = Gaussienne(degre=degre, L=L)
    gaussienne.entrainement(ds_train)
    
    scores.append(
        Score(
            modele=gaussienne, 
            score_entrainement=gaussienne.erreur_quadratique(ds_train),
            score_selection=gaussienne.erreur_quadratique(ds_select)
        )
    )
    

CPU times: user 1.74 s, sys: 0 ns, total: 1.74 s
Wall time: 1.74 s


In [24]:
scores

[Score(modele=Gaussienne(degre=10, L=5.0), score_entrainement=0.02769062244786264, score_selection=0.020889107010881057),
 Score(modele=Gaussienne(degre=13, L=5.0), score_entrainement=0.008037099685217394, score_selection=0.007250482865241726),
 Score(modele=Gaussienne(degre=16, L=5.0), score_entrainement=0.0074030697743405465, score_selection=0.007035008425582856),
 Score(modele=Gaussienne(degre=19, L=5.0), score_entrainement=0.007147519268674039, score_selection=0.007842887668348826),
 Score(modele=Gaussienne(degre=22, L=5.0), score_entrainement=0.006967243303264961, score_selection=0.007867298704059474),
 Score(modele=Gaussienne(degre=25, L=5.0), score_entrainement=0.006620152941758873, score_selection=0.008943108614871043),
 Score(modele=Gaussienne(degre=28, L=5.0), score_entrainement=0.006474163086377383, score_selection=0.010674649069289511),
 Score(modele=Gaussienne(degre=31, L=5.0), score_entrainement=0.0062491110867074675, score_selection=0.01439234873750922),
 Score(modele=Ga

In [25]:
meilleur = min(scores, key=lambda s: s.score_selection)
meilleur

Score(modele=Gaussienne(degre=16, L=5.0), score_entrainement=0.0074030697743405465, score_selection=0.007035008425582856)

In [26]:
modele_selectionne = meilleur.modele

In [27]:
modele_selectionne.entrainement(ds=ds_reste)

In [28]:
print(f"""
Meilleur modèle
erreur training: {modele_selectionne.erreur_quadratique(ds_reste)}
erreur_test    : {modele_selectionne.erreur_quadratique(ds_test)}
""")



Meilleur modèle
erreur training: 0.007158431104571609
erreur_test    : 0.012121310981968213



On des ordres de grandeur similaire entre les deux erreurs, on n'est à priori pas en surapprentissage.

**EXERCICE** Modifier la phase de sélection de la façon suivante:

1. `ds_reste` est découper en 5 parties égales
2. On répète 5 fois la procédure: j'entraine sur 4 partie et je stocke l'erreur sur la 5 ème
3. Le score du modele est alors la moyenne de ces 5 erreurs

C'est ce qu'on appelle la **cross_validation**


In [34]:
def fusion(dsa: Dataset, dsb: Dataset) -> Dataset:
    xk = list()
    xk.extend(dsa.xk)
    xk.extend(dsb.xk)
    yk = list()
    yk.extend(dsa.yk)
    yk.extend(dsb.yk)    
    return Dataset(xk=xk, yk=yk)
    
def decoupage(ds: Dataset) -> Dataset:
    taille_bloc = len(ds.xk) // 3
    return [
        Dataset(
            xk=ds.xk[: taille_bloc], 
            yk=ds.yk[: taille_bloc]
        ),
        Dataset(
            xk=ds.xk[taille_bloc : 2 * taille_bloc], 
            yk=ds.yk[taille_bloc : 2 * taille_bloc]
        ),
        Dataset(
            xk=ds.xk[2 * taille_bloc :], 
            yk=ds.yk[2 * taille_bloc :]
        ),

    ]

In [35]:
def rotation(ds: Dataset) -> Iterable[tuple[Dataset, Dataset]]:
    ds1, ds2, ds3 = decoupage(ds, nb_parties)
    yield fusion(ds2, ds3), ds1
    yield fusion(ds1, ds3), ds2
    yield fusion(ds1, ds2), ds3
    

In [36]:
def cross_validation(ds: Dataset, modele: Gaussienne) -> Score:
    erreurs_entrainement = list()
    erreurs_selection = list()
    for ds_train, ds_select in rotation(ds):
        modele.entrainement(ds_train)
        erreurs_entrainement.append(modele.erreur_quadratique(ds_train))
        erreurs_selection.append(modele.erreur_quadratique(ds_select))
    return Score(
        modele=modele,
        score_entrainement=sum(erreurs_entrainement) / len(erreurs_entrainement),
        score_selection=sum(erreurs_selection) / len(erreurs_selection),
    )

In [37]:
%%time
scores = [
    cross_validation(ds=ds_reste, modele=Gaussienne(degre=degre, L=L)) 
    for degre in range(10, 100, 3)
]

    

CPU times: user 2.65 s, sys: 0 ns, total: 2.65 s
Wall time: 2.65 s


In [38]:
scores

[Score(modele=Gaussienne(degre=10, L=5.0), score_entrainement=0.02572294457636683, score_selection=0.02777279025525284),
 Score(modele=Gaussienne(degre=13, L=5.0), score_entrainement=0.007194049832208914, score_selection=0.01067021960159355),
 Score(modele=Gaussienne(degre=16, L=5.0), score_entrainement=0.006563930956408712, score_selection=0.01053717436315546),
 Score(modele=Gaussienne(degre=19, L=5.0), score_entrainement=0.006329959266464675, score_selection=0.010762361683820597),
 Score(modele=Gaussienne(degre=22, L=5.0), score_entrainement=0.006111317056050593, score_selection=0.011203617526101535),
 Score(modele=Gaussienne(degre=25, L=5.0), score_entrainement=0.005861373356086028, score_selection=0.011262231948445183),
 Score(modele=Gaussienne(degre=28, L=5.0), score_entrainement=0.005695595584560627, score_selection=0.01162930474563109),
 Score(modele=Gaussienne(degre=31, L=5.0), score_entrainement=0.005473151965672818, score_selection=0.01322014745157342),
 Score(modele=Gaussien

In [39]:
meilleur = min(scores, key=lambda s: s.score_selection)
meilleur

Score(modele=Gaussienne(degre=16, L=5.0), score_entrainement=0.006563930956408712, score_selection=0.01053717436315546)

In [40]:
meilleur.modele.entrainement(ds=ds_reste)
print(f"""
Meilleur modèle
erreur training: {meilleur.modele.erreur_quadratique(ds_reste)}
erreur_test    : {meilleur.modele.erreur_quadratique(ds_test)}
""")



Meilleur modèle
erreur training: 0.007158431104571609
erreur_test    : 0.012121310981968211



**EXERCICE** 

1. Adapter le code de `cross_validation` pour prendre en argument le nombre de parties du découpage, et les fonctions sous jacentes.
2. Adapter la classe `Dataset` pour faciliter l'implémentation des méthodes précédentes en n'ayant plus à travailler sur les listes xk et yk directement.


**EXERCICE**

Coder une classe `RN` de meme interface que `Gaussienne` mais représentant un réseau de neurones à une couche cachée.

Indication: déterminer les formules mathématiques explicites entre les différentes variables quand l'entrée est vectorielle puis regarder la documentation de la fonction `np.einsum`.

In [76]:
class RN:
    def __init__(self, n: int):
        self.n = n
        self.alpha = np.zeros(shape=(n, ))
        self.beta = np.zeros(shape=(n, ))
        self.gamma = np.zeros(shape=(n, ))
        self.beta2 = 0.
        
    def sigma(self, z):
        return 1. / (1. + np.exp(-z))
        
    def __repr__(self):
        return f"RN(n={len(self.alpha)})"
        
    def __call__(self, x: np.array) -> np.array:
        if isinstance(x, float):
            x = np.array([x])
        c = self.sigma(np.einsum("k,i->ik", self.alpha , x) + self.beta)
        return self.sigma(np.einsum("k,ik->i", self.gamma, c) + self.beta2)
    
        
    def affichage(self, repere, couleur="magenta"):
        xg, xd = repere.get_xlim()
        xs = np.linspace(xg, xd, 500)
        ys = self(xs)
        repere.plot(xs, ys, label="prédicteur", color=couleur)
        
    def erreur_quadratique(self, ds: Dataset) -> float:
        return np.sum((self(ds.xk) - ds.yk)**2) / len(ds.xk)
    
    def entrainement(self, ds: Dataset):
        def calcule_residus(cs):
            self.alpha = cs[:self.n]
            self.beta = cs[self.n: 2 * self.n]
            self.gamma = cs[2 * self.n: 3 * self.n ]
            self.beta2 = cs[-1]
            return self(ds.xk) - ds.yk
        
        opti_result = least_squares(
            fun=calcule_residus,
            x0=np.zeros(shape=(3*self.n+1,)),
            max_nfev=100000,
        )
        if not opti_result.success:
            print("Entrainement Sans succès",)
            print(opti_result.message)
        self.coefficients = opti_result.x



In [77]:
rn = RN(6)

In [78]:
rn(np.linspace(0, 1, 10))

array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5])

In [79]:
rn(0.5)

array([0.5])

In [80]:
L = 5.
taille_echantillon = 200

reel = Realite(L=L)
ds = reel.ds_raisonnable(N=taille_echantillon, epsilon=0.1)

In [81]:
ds_reste, ds_test = train_test_split(ds, proportion=0.8)

In [82]:
%%time
scores = [
    cross_validation(ds=ds_reste, modele=RN(n=n)) 
    for n in range(2, 16)
]

KeyboardInterrupt: 